In [2]:
import pandas as pd

In [63]:
df=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\최종전.csv')
df.head()
df.columns=[col.replace('/12','') for col in df.columns]
df.to_csv('최종전_12제거.csv',index=False, encoding='utf-8-sig')

In [67]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv(r"C:\Users\bitcamp\OneDrive\Desktop\pythontraning\최종전_12제거.csv", encoding='utf-8-sig')  # 경로는 실제 파일명에 맞게 수정하세요

# 매출액 컬럼이 비어 있는 경우 다른 수익 항목으로 채우기
for year in ['2022', '2023', '2024']:
    sales_col = f"매출액_{year}"
    interest_col = f"이자수익_{year}"
    insurance_col = f"보험료수익_{year}"
    business_col = f"영업수익_{year}"
    real_col=f"순영업수익_{year}"
    
    if sales_col in df.columns:
        # 순서대로 존재하는 보조 수익 컬럼에서 채워 넣기
        if interest_col in df.columns:
            df[sales_col] = df[sales_col].fillna(df[interest_col])
        if insurance_col in df.columns:
            df[sales_col] = df[sales_col].fillna(df[insurance_col])
        if business_col in df.columns:
            df[sales_col] = df[sales_col].fillna(df[business_col])
        if real_col in df.columns:
            df[sales_col] = df[sales_col].fillna(df[real_col])

# 결과 저장 (선택)
df.to_csv("최종전_매출액보정.csv", index=False, encoding='utf-8-sig')

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# ✅ 1. 코스피 종목코드 불러오기
df_codes = pd.read_excel(r"C:\Users\bitcamp\OneDrive\Desktop\pythontraning\DART_재무_코스피전체_2024기준 (1).xlsx")
codes = df_codes['종목코드'].dropna().unique()

# ✅ 2. 드라이버 설정
options = Options()
# options.add_argument('--headless')  # 크롤링 중 문제 생기면 주석 처리
options.add_argument('--window-size=1920,1080')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# ✅ 3. 결과 누적용 리스트
results = []

for i, code in enumerate(codes):
    try:
        code_fmt = f"A{str(code).zfill(6)}"
        url = f"https://comp.fnguide.com/SVO2/ASP/SVD_shareanalysis.asp?pGB=1&gicode={code_fmt}"
        driver.get(url)
        time.sleep(2)

        # 테이블 찾기
        table = driver.find_element(By.ID, 'dataTable')
        html = table.get_attribute('outerHTML')
        df = pd.read_html(html)[0]
        df.insert(0, '종목코드', code_fmt)

        results.append(df)
        print(f"✅ [{i+1}/{len(codes)}] {code_fmt} 수집 완료")

    except Exception as e:
        print(f"❌ [{i+1}/{len(codes)}] {code_fmt} 실패: {e}")
        continue

# ✅ 4. 드라이버 종료
driver.quit()

# ✅ 5. 통합 저장
if results:
    final_df = pd.concat(results, ignore_index=True)
    final_df.to_csv("코스피_주주구분현황_전체.csv", index=False, encoding='utf-8-sig')
    print("📁 전체 저장 완료 → '코스피_주주구분현황_전체.csv'")
else:
    print("❗ 수집된 데이터가 없습니다.")



In [19]:
tb=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\코스피_주주구분현황_전체.csv')
tb.head(12)

,종목코드,주주구분,대표 주주수,보통주,지분율,최종변동일
0,A095570,최대주주등 (본인+특별관계자),1.0,26794111.0,59.21,2025/03/26
1,A095570,10%이상주주 (본인+특별관계자),NaN,NaN,NaN,NaN
2,A095570,5%이상주주 (본인+특별관계자),NaN,NaN,NaN,NaN
3,A095570,"임원 (5%미만 중, 임원인자)",10.0,143916.0,0.32,2024/07/09
4,A095570,자기주식 (자사주+자사주신탁),1.0,501709.0,1.11,2024/08/06
5,A095570,우리사주조합,1.0,187425.0,0.41,2023/12/31
6,A006840,최대주주등 (본인+특별관계자),1.0,8635613.0,65.19,2025/03/31
7,A006840,10%이상주주 (본인+특별관계자),NaN,NaN,NaN,NaN
8,A006840,5%이상주주 (본인+특별관계자),NaN,NaN,NaN,NaN
9,A006840,"임원 (5%미만 중, 임원인자)",1.0,1693.0,0.01,2024/03/22


In [20]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("코스피_주주구분현황_전체.csv", encoding='utf-8-sig')

# 필요한 컬럼만 선택
df_sel = df[['종목코드', '주주구분', '보통주', '지분율']].copy()

# 결측치 모두 숫자형으로 통일
df_sel['보통주'] = pd.to_numeric(df_sel['보통주'], errors='coerce')
df_sel['지분율'] = pd.to_numeric(df_sel['지분율'], errors='coerce')

# 피벗: 주주구분 별 지표를 컬럼으로
pivot_df = df_sel.pivot_table(
    index='종목코드',
    columns='주주구분',
    values=['보통주', '지분율']
)

# 컬럼 정리 (멀티인덱스 제거)
pivot_df.columns = [f"{col2}_{col1}" for col1, col2 in pivot_df.columns]
pivot_df.reset_index(inplace=True)

# 저장
pivot_df.to_csv("코스피_주주구분현황_가로변환.csv", index=False, encoding='utf-8-sig')
print("✅ 6줄 → 1줄 구조로 변환 완료")

✅ 6줄 → 1줄 구조로 변환 완료


In [22]:
tb1=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\코스피_주주구분현황_가로변환.csv')
tb1

,종목코드,10%이상주주 (본인+특별관계자)_보통주,5%이상주주 (본인+특별관계자)_보통주,개인/법인_보통주,기관투자자_보통주,우리사주조합_보통주,"임원 (5%미만 중, 임원인자)_보통주",자기주식 (자사주+자사주신탁)_보통주,주요외국인주주_보통주,최대주주등 (본인+특별관계자)_보통주,...,10%이상주주 (본인+특별관계자)_지분율,5%이상주주 (본인+특별관계자)_지분율,개인/법인_지분율,기관투자자_지분율,우리사주조합_지분율,"임원 (5%미만 중, 임원인자)_지분율",자기주식 (자사주+자사주신탁)_지분율,주요외국인주주_지분율,최대주주등 (본인+특별관계자)_지분율,해외DR_지분율
0,A000020,NaN,NaN,NaN,NaN,1100582.0,NaN,213540.0,NaN,9397282.0,...,NaN,NaN,NaN,NaN,3.94,NaN,0.76,NaN,33.64,NaN
1,A000040,NaN,NaN,NaN,NaN,NaN,909.0,11608.0,NaN,28313437.0,...,NaN,NaN,NaN,NaN,NaN,0.00,0.02,NaN,47.08,NaN
2,A000050,3230860.0,NaN,NaN,NaN,470.0,NaN,2500000.0,NaN,16705710.0,...,11.78,NaN,NaN,NaN,0.00,NaN,9.12,NaN,60.94,NaN
3,A000070,NaN,447474.0,NaN,NaN,986.0,NaN,1128726.0,NaN,3590842.0,...,NaN,5.22,NaN,NaN,0.01,NaN,13.18,NaN,41.93,NaN
4,A000080,NaN,4219117.0,NaN,NaN,135938.0,42398.0,1484184.0,NaN,37740562.0,...,NaN,6.02,NaN,NaN,0.19,0.06,2.12,NaN,53.81,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,A487570,NaN,NaN,NaN,NaN,NaN,NaN,19700.0,NaN,2149927.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.53,NaN,57.70,NaN
787,A489790,NaN,6935181.0,NaN,NaN,NaN,2109.0,110848.0,NaN,17144731.0,...,NaN,13.74,NaN,NaN,NaN,0.00,0.22,NaN,33.96,NaN
788,A499790,NaN,1306133.0,NaN,NaN,NaN,NaN,46221.0,NaN,11625619.0,...,NaN,6.59,NaN,NaN,NaN,NaN,0.23,NaN,58.62,NaN
789,A900140,NaN,NaN,NaN,NaN,NaN,NaN,299363.0,NaN,69100516.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.17,NaN,40.27,NaN


In [7]:
pip install --upgrade selenium


   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------  9.2/9.4 MB 54.3 MB/s eta 0:00:01
   ---------------------------------------  9.2/9.4 MB 54.3 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 21.8 MB/s eta 0:00:00
  Attempting uninstall: selenium
    Found existing installation: selenium 4.30.0
    Uninstalling selenium-4.30.0:
      Successfully uninstalled selenium-4.30.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [70]:
import pandas as pd

df1 = pd.read_csv(r"C:\Users\bitcamp\OneDrive\Desktop\pythontraning\최종전_매출액보정.csv")
df2 = pd.read_csv(r"C:\Users\bitcamp\OneDrive\Desktop\pythontraning\업종명.csv")
df2.rename(columns={'종목명':"기업명"},inplace=True)

merged=pd.merge(df1,df2,on='기업명',how='left')
merged.to_csv("최종전_업종명추가.csv", index=False,encoding='utf-8-sig')





# # 예: '종목코드' 또는 '날짜'가 공통된 컬럼일 경우
# merged = pd.merge(df1, df2, on="종목코드", how="left")  # 또는 how="inner"

# # 저장
# merged.to_csv("합쳐진파일1.csv", index=False,encoding='utf-8-sig')

In [81]:
df3=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\최종전_업종명추가.csv')
df3.head()
df3.rename(columns={'종목코드_x':"종목코드"},inplace=True)
df3.drop(['보험료수익_2022', '보험료수익_2023', '보험료수익_2024', '이자수익_2022','이자수익_2023', '이자수익_2024', '영업수익_2022', '영업수익_2023', '영업수익_2024','순영업수익_2022', '순영업수익_2023', '순영업수익_2024','시장구분','종목코드_y','종가','대비','등락률'],axis=1,inplace=True)
df3.to_csv('최종전_전처리.csv',index=False, encoding='utf-8-sig')
df3.head()

,기업명,종목코드,매출액_2022,영업이익_2022,당기순이익_2022,자산총계_2022,부채총계_2022,ROE_2022,EPS(원)_2022,DPS(원)_2022,...,자산총계_2024,부채총계_2024,ROE_2024,EPS(원)_2024,DPS(원)_2024,PER_2024,PBR_2024,배당수익률_2024,업종명,시가총액
0,AJ네트웍스,A095570,11937.0,764.0,87.0,14834.0,11197.0,2.45,190.0,270.0,...,17178.0,12740.0,5.04,481.0,270.0,8.96,0.44,6.27,일반서비스,1.719605e+11
1,AK홀딩스,A006840,37880.0,-218.0,-1249.0,49854.0,37220.0,-11.21,-4811.0,200.0,...,53369.0,40918.0,-5.28,-2199.0,400.0,NaN,0.24,4.15,기타금융,1.396293e+11
2,BGF,A027410,4350.0,612.0,262.0,19380.0,2015.0,1.5,247.0,110.0,...,23010.0,3199.0,5.43,963.0,130.0,3.58,0.19,3.77,기타금융,3.455376e+11
3,BGF리테일,A282330,76158.0,2524.0,1935.0,29462.0,19836.0,21.84,11198.0,4100.0,...,34026.0,22185.0,17.29,11295.0,4100.0,9.07,1.49,4.00,유통,1.754316e+12
4,BNK금융지주,A138930,47790.0,11561.0,8332.0,1369326.0,1261427.0,8.19,2408.0,625.0,...,1524705.0,1413229.0,6.96,2266.0,650.0,4.56,0.31,6.29,기타금융,3.301637e+12


In [80]:
df_final = pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\최종전_전처리.csv')

# 하나라도 NaN이 있는 열만 필터링
cols_with_nan = df_final.columns[df_final.isnull().any()]
cols_with_nan

Index(['매출액_2022', '영업이익_2022', '당기순이익_2022', '자산총계_2022', '부채총계_2022',
       'ROE_2022', 'EPS(원)_2022', 'DPS(원)_2022', 'PER_2022', 'PBR_2022',
       '배당수익률_2022', '매출액_2023', '영업이익_2023', '당기순이익_2023', '자산총계_2023',
       '부채총계_2023', 'ROE_2023', 'EPS(원)_2023', 'DPS(원)_2023', 'PER_2023',
       'PBR_2023', '배당수익률_2023', 'ROE_2024', 'DPS(원)_2024', 'PER_2024',
       'PBR_2024', '배당수익률_2024', '업종명', '시가총액'],
      dtype='object')

In [ ]:
df4=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\data_5152_20250512 (1).xlsx',encoding='cp949')
df5=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\전처리1.csv')

merged = pd.merge(df1, df2, on="종목코드", how="left")  # 또는 how="inner"

# 저장
merged.to_csv("전처리2.csv", index=False,encoding='utf-8-sig')



# 종목코드에서 코드 앞 A지우기, 기업명_X고치기, 업종구분명 추가

UnicodeDecodeError: 'cp949' codec can't decode byte 0xcf in position 43: illegal multibyte sequence

In [2]:
import pandas as pd

# 엑셀 파일 경로
excel_file = r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\data_5152_20250512 (1).xlsx'

# 엑셀 파일의 첫 번째 시트를 읽기
df = pd.read_excel(excel_file)

# CSV로 저장
csv_file = '파일명.csv'
df.to_csv(csv_file, index=False, encoding='utf-8-sig')  # 한글이 있을 경우 'utf-8-sig' 권장

c:\Users\bitcamp\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df4=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\파일명.csv')
df5=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\전처리1.csv')

merged = pd.merge(df4, df5, on="기업명", how="left")  # 또는 how="inner"

# 저장
merged.to_csv("전처리2.csv", index=False,encoding='utf-8-sig')

In [30]:
df5=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\전처리1.csv')
df5.columns
df6=df5.rename(columns={'기업명_x':"종목명"})
df6.to_csv('전처리3.csv',index=False,encoding='utf-8-sig')

In [ ]:
df10=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\업종명.csv')
df11=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\전처리3.csv')

merged = pd.merge(df10, df11, on="종목명", how="left")  # 또는 how="inner"

# 저장
merged.to_csv("전처리2.csv", index=False,encoding='utf-8-sig')


In [55]:
df12=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\전처리2.csv')
df12.drop(['시장구분','종가','대비','등락률','시가총액','종목코드_y'],axis=1,inplace=True)
df12.rename(columns={'종목코드_x':'종목코드'},inplace=True)
df12.to_csv('전처리4.csv',index=False,encoding='utf-8-sig')
df12.head()

,종목코드,종목명,업종명,매출액_2022,영업이익_2022,당기순이익_2022,자산총계_2022,부채총계_2022,자본총계_2022,ROE_2022,...,ROE_2024,PER_2024,PBR_2024,배당수익률_2024,EPS(원)_2022,DPS(원)_2022,EPS(원)_2023,DPS(원)_2023,EPS(원)_2024,DPS(원)_2024
0,095570,AJ네트웍스,일반서비스,11937.0,764.0,87.0,14834.0,11197.0,3637.0,2.45,...,5.04,8.96,0.44,6.27,190.0,270.0,357.0,270.0,481.0,270.0
1,006840,AK홀딩스,기타금융,37880.0,-218.0,-1249.0,49854.0,37220.0,12634.0,-11.21,...,-5.28,NaN,0.24,4.15,-4811.0,200.0,2600.0,200.0,-2199.0,400.0
2,027410,BGF,기타금융,4350.0,612.0,262.0,19380.0,2015.0,17365.0,1.5,...,5.43,3.58,0.19,3.77,247.0,110.0,813.0,120.0,963.0,130.0
3,282330,BGF리테일,유통,76158.0,2524.0,1935.0,29462.0,19836.0,9626.0,21.84,...,17.29,9.07,1.49,4.00,11198.0,4100.0,11331.0,4100.0,11295.0,4100.0
4,138930,BNK금융지주,기타금융,47790.0,11561.0,8332.0,1369326.0,1261427.0,107899.0,8.19,...,6.96,4.56,0.31,6.29,2408.0,625.0,1972.0,510.0,2266.0,650.0


In [ ]:
df13=pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\전처리4.csv')
df13['종목코드'] = df13['종목코드'].astype(str).str.zfill(6)
df13.to_csv('전처리5.csv',index=False,encoding='utf-8-sig')
df13.columns



Index(['종목코드', '종목명', '업종명', '매출액_2022', '영업이익_2022', '당기순이익_2022',
       '자산총계_2022', '부채총계_2022', '자본총계_2022', 'ROE_2022', 'PER_2022',
       'PBR_2022', '배당수익률_2022', '매출액_2023', '영업이익_2023', '당기순이익_2023',
       '자산총계_2023', '부채총계_2023', '자본총계_2023', 'ROE_2023', 'PER_2023',
       'PBR_2023', '배당수익률_2023', '매출액_2024', '영업이익_2024', '당기순이익_2024',
       '자산총계_2024', '부채총계_2024', '자본총계_2024', 'ROE_2024', 'PER_2024',
       'PBR_2024', '배당수익률_2024', 'EPS(원)_2022', 'DPS(원)_2022', 'EPS(원)_2023',
       'DPS(원)_2023', 'EPS(원)_2024', 'DPS(원)_2024'],
      dtype='object')

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# 📌 CSV 불러오기
df = pd.read_csv(r'C:\Users\bitcamp\OneDrive\Desktop\pythontraning\빈값.csv', encoding='utf-8-sig')

# 📌 종목코드 정리 함수
def format_code(x):
    x = str(x)
    if x.isdigit():
        return 'A' + x.zfill(6)  # 숫자면 6자리 + 'A' 접두사
    return None  # 문자가 포함된 종목코드는 제거

df['종목코드_정리'] = df['종목코드'].apply(format_code)

# 📌 문자 포함 종목코드 제거
df = df.dropna(subset=['종목코드_정리'])

# 📌 크롤링 대상 DataFrame 구성
kospi_df = df[['회사명', '종목코드_정리']].rename(columns={'회사명': '기업명', '종목코드_정리': '종목코드'}).reset_index(drop=True)

# 📌 셀레니움 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

rows = []

# 📌 크롤링 루프
for idx, row in kospi_df.iterrows():
    name = row['기업명']
    code = row['종목코드']
    url = f"https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode={code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701"
    print(f"▶ [{idx+1}/{len(kospi_df)}] {name} 처리 중...")

    try:
        driver.get(url)
        time.sleep(1.5)

        table = driver.find_element(By.ID, 'highlight_D_A')
        html = table.get_attribute('outerHTML')
        df_fin = pd.read_html(html, header=1)[0]

        if isinstance(df_fin.columns, pd.MultiIndex):
            df_fin.columns = df_fin.columns.get_level_values(-1)

        df_fin.rename(columns={df_fin.columns[0]: '항목'}, inplace=True)
        df_fin['항목'] = df_fin['항목'].str.strip()

        # 📌 필요한 항목 필터링
        target_rows = ['매출액','영업이익','당기순이익','자산총계','부채총계','ROE','PER','PBR','배당수익률','EPS(원)','DPS(원)']
        target_columns = ['항목', '2022/12', '2023/12', '2024/12']
        df_filtered = df_fin[df_fin['항목'].isin(target_rows)][target_columns]

        # 📌 가로 펼치기
        df_melted = df_filtered.melt(id_vars=['항목'], var_name='연도', value_name='값')
        df_melted['컬럼명'] = df_melted['항목'] + "_" + df_melted['연도']
        df_flat = df_melted[['컬럼명', '값']].set_index('컬럼명').T
        df_flat['기업명'] = name
        df_flat['종목코드'] = code

        rows.append(df_flat)

    except Exception as e:
        print(f"❌ {name} 실패: {e}")
        continue

driver.quit()

# 📌 병합 및 저장
final_df = pd.concat(rows, ignore_index=True)
cols = ['기업명', '종목코드'] + [col for col in final_df.columns if col not in ['기업명', '종목코드']]
final_df = final_df[cols]

final_df.to_csv("eps_dps1.csv", index=False, encoding='utf-8-sig')
print("✅ 전체 코스피 기업 재무요약 저장 완료!")



In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

options = Options()
# options.add_argument('--headless')
options.add_argument('--window-size=1920,1080')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 예시 기업 (SK하이닉스)
code = "A000660"
name = "SK하이닉스"
url = f"https://comp.fnguide.com/SVO2/ASP/SVD_Corp.asp?pGB=1&gicode={code}"

try:
    driver.get(url)
    time.sleep(3)

    # 테이블 선택: '매출비중 추이' (caption 확인됨)
    table = driver.find_element(By.CSS_SELECTOR, 'table.us_table_ty1.table-hb2.h_fix.zigbg_no')
    html = table.get_attribute('outerHTML')
    df = pd.read_html(html)[0]

    # 종목 정보 추가
    df.insert(0, '종목코드', code)
    df.insert(1, '종목명', name)

    # 저장
    df.to_csv(f"{name}_매출비중추이.csv", index=False, encoding='utf-8-sig')
    print("✅ 매출비중 추이 저장 완료")
    print(df.head())

except Exception as e:
    print(f"❌ 오류 발생: {e}")

finally:
    driver.quit()


C:\Users\bitcamp\AppData\Local\Temp\ipykernel_21692\3832604294.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]


✅ 매출비중 추이 저장 완료
      종목코드     종목명                 제품명  2021/12  2022/12  2023/12  2024/12
0  A000660  SK하이닉스  DRAM, NAND Flash 등      NaN      NaN      NaN    100.0
1  A000660  SK하이닉스              반도체 부문    100.0    100.0    100.0      NaN
2  A000660  SK하이닉스               기타(계)      NaN      NaN      NaN      NaN


In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# ① 종목코드 데이터 불러오기
df_codes = pd.read_excel(r"C:\Users\bitcamp\OneDrive\Desktop\pythontraning\DART_재무_코스피전체_2024기준 (1).xlsx")
codes = df_codes[['종목코드', '종목명']].dropna()

# ② Selenium 설정
options = Options()
options.add_argument('--headless')  # → 크롬 창 안 뜨도록
options.add_argument('--window-size=1920,1080')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# ③ 결과 저장용 리스트
results = []

for i, row in codes.iterrows():
    code = str(row['종목코드']).zfill(6)
    name = row['종목명']
    url = f"https://comp.fnguide.com/SVO2/ASP/SVD_Corp.asp?pGB=1&gicode=A{code}"

    try:
        driver.get(url)
        time.sleep(2)

        # 매출비중 테이블 (class 기반 탐색)
        table = driver.find_element(By.CSS_SELECTOR, 'table.us_table_ty1.table-hb2.h_fix.zigbg_no')
        html = table.get_attribute('outerHTML')
        df = pd.read_html(html)[0]

        df.insert(0, '종목코드', f"A{code}")
        df.insert(1, '종목명', name)
        results.append(df)
        print(f"✅ [{i+1}] {name} 크롤링 완료")

    except Exception as e:
        print(f"❌ [{i+1}] {name} 실패: {e}")
        continue

driver.quit()

# ④ 병합 후 저장
if results:
    final_df = pd.concat(results, ignore_index=True)
    final_df.to_csv("코스피_매출비중추이_전체.csv", index=False, encoding='utf-8-sig')
    print("📁 저장 완료: 코스피_매출비중추이_전체.csv")
else:
    print("❗ 크롤링된 데이터 없음")


C:\Users\bitcamp\AppData\Local\Temp\ipykernel_21692\2138220853.py:39: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]


✅ [1] AJ네트웍스 크롤링 완료


C:\Users\bitcamp\AppData\Local\Temp\ipykernel_21692\2138220853.py:39: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]


✅ [2] AK홀딩스 크롤링 완료


KeyboardInterrupt: 